In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

%matplotlib inline

train_perf = pd.read_csv('trainperf.csv')
perf['good_bad_flag'] = perf.good_bad_flag.replace({'Good':1,'Bad':0})

# PERFORMANCE

In [2]:
train_perf = pd.read_csv('trainperf.csv')
test_perf = pd.read_csv('testperf.csv')

In [3]:
 id = test_perf['customerid']  

In [4]:
frames = [train_perf,test_perf]
perf = pd.concat(frames)
perf.shape

(5818, 10)

In [6]:
perf['good_bad_flag'] = perf.good_bad_flag.replace({'Good':1,'Bad':0})
#train_perf.corr()

In [9]:
train_perf.isnull().sum()

customerid          0
systemloanid        0
loannumber          0
approveddate        0
creationdate        0
loanamount          0
totaldue            0
termdays            0
referredby       3781
good_bad_flag       0
dtype: int64

In [5]:
data = perf
data.head()

,approveddate,creationdate,customerid,good_bad_flag,loanamount,loannumber,referredby,systemloanid,termdays,totaldue
0,22:56.0,22:47.0,8a2a81a74ce8c05d014cfb32a0da1049,Good,30000,12,NaN,301994762,30,34500.0
1,04:41.0,04:18.0,8a85886e54beabf90154c0a29ae757c0,Good,15000,2,NaN,301965204,30,17250.0
2,52:57.0,52:51.0,8a8588f35438fe12015444567666018e,Good,20000,7,NaN,301966580,15,22250.0
3,00:41.0,00:35.0,8a85890754145ace015429211b513e16,Good,10000,3,NaN,301999343,15,11500.0
4,42:45.0,42:39.0,8a858970548359cc0154883481981866,Good,40000,9,NaN,301962360,30,44000.0


In [7]:
data['referredby'] = data['referredby'].fillna(0)
data['referredby'] = data['referredby'].apply(lambda x : 0 if x==0 else 1)

In [8]:
data['interest'] = data['totaldue'] - data['loanamount']

In [15]:
data[(data.good_bad_flag==0)].interest.mean()

3343.986344537815

In [9]:
data['loan_ratio'] = data.loanamount/data['loannumber']

In [10]:
data['termdays'] = data['termdays'].replace({15:1,30:2,60:3,90:4})

In [ ]:
count_class_1, count_class_0 = datas.good_bad_flag.value_counts()
df_class_0 = datas[datas['good_bad_flag']==0]
df_class_1 = datas[datas['good_bad_flag']==1]

In [ ]:
datas.good_bad_flag.value_counts()

In [ ]:
df_class_1_under = df_class_1.sample(count_class_0,replace=True)
datas= pd.concat([df_class_1_under,df_class_0], axis=0)
datas.shape

# DEMOGRAPHICS

In [11]:
train_demographics = pd.read_csv('traindemographics.csv')
test_demographics = pd.read_csv('testdemographics.csv')

In [12]:
frames = [train_demographics,test_demographics]

# Combine train & test
demographics = pd.concat(frames)
demographics.shape

(5833, 9)

In [13]:
demographics.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


In [14]:
print(train_demographics.shape,test_demographics.shape)

(4346, 9) (1487, 9)


In [15]:
demographics = demographics.drop_duplicates(subset='customerid',keep='last')

In [16]:
demographics['age'] = demographics['birthdate'].apply(lambda x: x.split().pop(0)[:10])
# Check
demographics['age'].head()

0    1973-10-10
1    1986-01-21
2    1987-04-01
3    1991-07-19
4    1982-11-22
Name: age, dtype: object

In [17]:
demographics['age'] = demographics['age'].apply(lambda x: x.split('-'))

# Convert all elements in each observation(list) from str to int
demographics['age'] = demographics['age'].apply(lambda x: [int(y) for y in x ])

# Convert each observation into a date format
demographics['age'] = demographics['age'].apply(lambda x : date(x[0], x[1], x[2]))

# Calculate Age!
demographics['age'] = demographics['age'].apply(lambda x:(date(2017,7,1) - x) // timedelta(days=365.2425) )

In [18]:
demographics = demographics.drop('level_of_education_clients',axis=1)
demographics = demographics.drop('bank_branch_clients',axis=1)


In [19]:
demographics.employment_status_clients.fillna(4,inplace=True)
demographics.replace({'employment_status_clients':{'Self-employed':1,'Student':2,'Unemployed':3,
                                                  'Retired':3,'Contract':1, 'Permanent':1}},inplace=True
                    )
demographics.replace({'bank_account_type':{'Current':1,'Other':2,'Savings':3}}, inplace=True)


In [20]:
data = pd.merge(data, demographics, how='outer', on=['customerid'])

# PREVIOUS LOANS

In [21]:
train_prevloans = pd.read_csv('trainprevloans.csv')
test_prevloans = pd.read_csv('testprevloans.csv')


In [22]:
frames = [train_prevloans,test_prevloans]
previous_loans = pd.concat(frames)
previous_loans.shape

(24090, 12)

In [39]:
previous_loans.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000


In [23]:
grouped = previous_loans[['customerid','loanamount','totaldue','loannumber','termdays']].groupby('customerid').sum()
grouped = grouped.reset_index()
grouped.columns = ['customerid','avg_loanamount','avg_totaldue','avg_loan','avg_term']
grouped.shape

(5801, 5)

In [ ]:
grouped = previous_loans[['customerid','loanamount','totaldue','loannumber','termdays']].groupby('customerid').mean()
grouped = grouped.reset_index()
grouped.columns = ['customerid','avg_loanamount','avg_totaldue','avg_loan','avg_term']
grouped.shape

In [24]:
data = pd.merge(data, grouped, how='outer', on=['customerid'])

In [25]:
previous_loans['firstduedate'] = previous_loans['firstduedate'].apply(lambda x: x.split().pop(0)[:10])
previous_loans['firstrepaiddate'] = previous_loans['firstrepaiddate'].apply(lambda x: x.split().pop(0)[:10])

In [26]:
previous_loans['firstduedate'] = previous_loans['firstduedate'].apply(lambda x: x.split('-'))

# Convert all elements in each observation(list) from str to int
previous_loans['firstduedate'] = previous_loans['firstduedate'].apply(lambda x: [int(y) for y in x ])

# Convert each observation into a date format
previous_loans['firstduedate'] = previous_loans['firstduedate'].apply(lambda x : date(x[0], x[1], x[2]))

# Split the  date (This converts each observation into a list)
previous_loans['firstrepaiddate'] = previous_loans['firstrepaiddate'].apply(lambda x: x.split('-'))

# Convert all elements in each observation(list) from str to int
previous_loans['firstrepaiddate'] = previous_loans['firstrepaiddate'].apply(lambda x: [int(y) for y in x ])

# Convert each observation into a date format
previous_loans['firstrepaiddate'] = previous_loans['firstrepaiddate'].apply(lambda x : date(x[0], x[1], x[2]))

# Calculate repayment_score
previous_loans['repayment_score'] = previous_loans['firstduedate'] - previous_loans['firstrepaiddate']

In [44]:
previous_loans.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate,repayment_score
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14,2016-09-01,13 days
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30,2017-05-26,4 days
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04,2017-04-26,-22 days
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24,2017-04-24,0 days
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03,2017-07-14,-11 days


In [27]:
previous_loans['repayment_score'] = previous_loans['repayment_score'].astype('str').apply(lambda x: int(x.strip(' ')[:2]))

In [30]:
grouped = previous_loans[['customerid','repayment_score','duration','loan_settling']].groupby('customerid').sum()
grouped = grouped.reset_index()

# HELPING HANDS

In [28]:
from datetime import datetime as dt
def duration(data):
    data['closeddate']= pd.to_datetime(data['closeddate'])
    data['approveddate']=pd.to_datetime(data['approveddate'])
    data['duration'] = data['closeddate'] - data['approveddate']
    data['duration'].apply(lambda x:x.days)
    data['duration'] = data['duration'].dt.total_seconds() / 86400
    return data


def interest_rate(data):
    data['interestrate'] = (data['interest']/data['loanamount']) * 100
    return data

def loan_settling(data):
    data.firstduedate= pd.to_datetime(data.firstduedate)
    data.closeddate= pd.to_datetime(data.closeddate)
    data['loan_settling'] = data['firstduedate'] - data['closeddate']
    data['loan_settling'].apply(lambda x:x.days)
    data['loan_settling'] = data['loan_settling'].dt.total_seconds() / 86400
    data['loan_settling'] = data['loan_settling'].map(lambda x:1 if x >= 0 else 0)
    return data

In [29]:
previous_loans = duration(previous_loans)

#data without the previous loan
data = interest_rate(data)

previous_loans = loan_settling(previous_loans)

In [ ]:
len(previous_loans['duration'])

In [31]:
data = pd.merge(data, grouped, how='outer', on=['customerid'])

In [52]:
data.head(8)

,approveddate,creationdate,customerid,good_bad_flag,loanamount,loannumber,referredby,systemloanid,termdays,totaldue,...,employment_status_clients,age,avg_loanamount,avg_totaldue,avg_loan,avg_term,interestrate,repayment_score,duration,loan_settling
0,22:56.0,22:47.0,8a2a81a74ce8c05d014cfb32a0da1049,1.0,30000,12,NaN,301994762,2,34500.0,...,1,45,200000.0,242900.0,66.0,330.0,15.00,79.0,329.606019,7.0
1,04:41.0,04:18.0,8a85886e54beabf90154c0a29ae757c0,1.0,15000,2,NaN,301965204,2,17250.0,...,1,31,NaN,NaN,NaN,NaN,15.00,NaN,NaN,NaN
2,52:57.0,52:51.0,8a8588f35438fe12015444567666018e,1.0,20000,7,NaN,301966580,1,22250.0,...,1,32,60000.0,70500.0,21.0,105.0,11.25,22.0,111.953519,3.0
3,00:41.0,00:35.0,8a85890754145ace015429211b513e16,1.0,10000,3,NaN,301999343,1,11500.0,...,1,39,20000.0,24500.0,3.0,45.0,15.00,-5.0,63.683981,0.0
4,42:45.0,42:39.0,8a858970548359cc0154883481981866,1.0,40000,9,NaN,301962360,2,44000.0,...,1,30,150000.0,188400.0,36.0,300.0,10.00,25.0,220.517616,8.0
5,46:24.0,46:18.0,8a8589f35451855401546b0738c42524,1.0,30000,8,NaN,301986516,3,39000.0,...,1,44,100000.0,124800.0,28.0,210.0,30.00,38.0,176.417894,4.0
6,25:33.0,25:28.0,8a858e095c59b91b015c5e5cea3719bc,1.0,10000,3,NaN,301972027,2,13000.0,...,1,28,20000.0,24500.0,3.0,45.0,30.00,5.0,40.004525,2.0
7,42:27.0,42:21.0,8a858e1158dc4d830158f7bde4f47ea7,1.0,30000,10,NaN,301994428,2,34500.0,...,NaN,29,180000.0,207000.0,45.0,210.0,15.00,9.0,202.611586,5.0


In [33]:
data[['avg_loanamount','avg_totaldue','avg_loan','avg_term']] = data[['avg_loanamount','avg_totaldue','avg_loan','avg_term']].fillna(0)

In [32]:
data['avg_loanamount'] = data['avg_loanamount'].fillna(data['avg_loanamount'].dropna().mean())
data['avg_totaldue'] = data['avg_totaldue'].fillna(data['avg_totaldue'].dropna().mean())
data['avg_loan'] = data['avg_loan'].fillna(data['avg_loan'].dropna().mean())
data['loan_settling'] = data['loan_settling'].fillna(data['loan_settling'].dropna().mean())
data['repayment_score'] = data['repayment_score'].fillna(data['repayment_score'].dropna().mean())

In [59]:
train_data['duration'] = train_data['duration'].fillna(train_data['duration'].dropna().mean())

In [ ]:
data[['repayment_score','duration','loan_settling']] = data[['repayment_score','duration','loan_settling']].fillna(0)

In [33]:
data['avg_interest'] = data['avg_totaldue'] - data['avg_loanamount']

# FEATURE ENGINEERING

In [35]:
step = 0.2
to_bin = lambda x: np.floor(x / step) * step
data["latbin"] = data['latitude_gps'].map(to_bin)
data["lonbin"] = data['longitude_gps'].map(to_bin)
data.head()

,approveddate,creationdate,customerid,good_bad_flag,loanamount,loannumber,referredby,systemloanid,termdays,totaldue,...,avg_totaldue,avg_loan,avg_term,interestrate,repayment_score,duration,loan_settling,avg_interest,latbin,lonbin
0,22:56.0,22:47.0,8a2a81a74ce8c05d014cfb32a0da1049,1.0,30000,12,0,301994762,2,34500.0,...,242900.000000,66.000000,330.0,15.00,79.000000,329.606019,7.000000,42900.000000,6.4,3.4
1,04:41.0,04:18.0,8a85886e54beabf90154c0a29ae757c0,1.0,15000,2,0,301965204,2,17250.0,...,81038.874556,17.402172,NaN,15.00,14.699707,NaN,2.561627,12747.373091,7.2,3.8
2,52:57.0,52:51.0,8a8588f35438fe12015444567666018e,1.0,20000,7,0,301966580,1,22250.0,...,70500.000000,21.000000,105.0,11.25,22.000000,111.953519,3.000000,10500.000000,10.2,11.0
3,00:41.0,00:35.0,8a85890754145ace015429211b513e16,1.0,10000,3,0,301999343,1,11500.0,...,24500.000000,3.000000,45.0,15.00,-5.000000,63.683981,0.000000,4500.000000,7.4,3.8
4,42:45.0,42:39.0,8a858970548359cc0154883481981866,1.0,40000,9,0,301962360,2,44000.0,...,188400.000000,36.000000,300.0,10.00,25.000000,220.517616,8.000000,38400.000000,9.0,7.4


In [33]:
datas= data.drop(['approveddate','creationdate','referredby','avg_term','customerid',
                  'birthdate','bank_name_clients','termdays','systemloanid'],axis=1)

In [34]:
ntrain = train_perf.shape[0]
ntest = test_perf.shape[0]
train = datas[:ntrain]
test = datas[ntrain:]

In [35]:
train_perf['good_bad_flag'] = train_perf.good_bad_flag.replace({'Good':1,'Bad':0})

In [36]:
train_target = train_perf.good_bad_flag.values

In [38]:
train.corr()

,good_bad_flag,loanamount,loannumber,totaldue,interest,loan_ratio,bank_account_type,longitude_gps,latitude_gps,age,avg_loanamount,avg_totaldue,avg_loan,interestrate,repayment_score,duration,loan_settling
good_bad_flag,1.000000,0.115267,0.090487,0.108357,0.032540,0.009665,-0.092743,0.008264,-0.018001,0.056248,0.089439,0.089801,0.076447,-0.131043,0.116407,0.043536,0.148084
loanamount,0.115267,1.000000,0.833318,0.994392,0.665706,-0.095656,-0.375955,-0.010007,0.026360,0.010422,0.855189,0.859866,0.765739,-0.518592,0.574086,0.756337,0.766016
loannumber,0.090487,0.833318,1.000000,0.815303,0.460606,-0.554393,-0.449131,-0.011735,0.011899,-0.007496,0.945555,0.949338,0.946639,-0.535945,0.603949,0.912393,0.850323
totaldue,0.108357,0.994392,0.815303,1.000000,0.740890,-0.078995,-0.368692,-0.010115,0.023537,0.014936,0.830172,0.836647,0.740071,-0.443280,0.559153,0.749735,0.746017
interest,0.032540,0.665706,0.460606,0.740890,1.000000,0.050094,-0.213903,-0.007823,-0.001327,0.039198,0.426624,0.442612,0.358949,0.165649,0.299529,0.486706,0.399140
loan_ratio,0.009665,-0.095656,-0.554393,-0.078995,0.050094,1.000000,0.243994,0.002338,0.018525,0.028319,-0.398930,-0.404913,-0.452110,0.190799,-0.197152,-0.540283,-0.378432
bank_account_type,-0.092743,-0.375955,-0.449131,-0.368692,-0.213903,0.243994,1.000000,0.004168,0.010560,-0.033349,-0.434506,-0.439713,-0.416909,0.230445,-0.214646,-0.474448,-0.346870
longitude_gps,0.008264,-0.010007,-0.011735,-0.010115,-0.007823,0.002338,0.004168,1.000000,-0.211244,0.002104,-0.007919,-0.007666,-0.013049,0.010291,-0.010195,-0.005358,-0.004134
latitude_gps,-0.018001,0.026360,0.011899,0.023537,-0.001327,0.018525,0.010560,-0.211244,1.000000,-0.025418,0.010935,0.010275,0.015175,-0.013779,0.026957,0.000583,0.025892
age,0.056248,0.010422,-0.007496,0.014936,0.039198,0.028319,-0.033349,0.002104,-0.025418,1.000000,0.002126,0.002012,-0.001795,0.040329,-0.020123,-0.001724,-0.000813


In [40]:
trains = train.drop(['good_bad_flag','employment_status_clients',
                     'avg_loan','loan_ratio'], axis=1)
#'totaldue','avg_totaldue'

In [42]:
tests= test.drop(['good_bad_flag','loan_ratio','employment_status_clients'
                    ,'avg_loan'], axis=1)

In [43]:
trains.columns

Index(['loanamount', 'loannumber', 'totaldue', 'interest', 'bank_account_type',
       'longitude_gps', 'latitude_gps', 'age', 'avg_loanamount',
       'avg_totaldue', 'interestrate', 'repayment_score', 'duration',
       'loan_settling'],
      dtype='object')

In [57]:
#train_data = pd.get_dummies(trains)
train_data.isnull().sum()

loanamount           0
loannumber           0
totaldue             0
interest             0
bank_account_type    0
longitude_gps        0
latitude_gps         0
age                  0
avg_loanamount       0
avg_totaldue         0
interestrate         0
repayment_score      0
duration             9
loan_settling        0
dtype: int64

In [51]:
#test_data = pd.get_dummies(tests)
test_data.isnull().sum()

loanamount           0
loannumber           0
totaldue             0
interest             0
bank_account_type    0
longitude_gps        0
latitude_gps         0
age                  0
avg_loanamount       0
avg_totaldue         0
interestrate         0
repayment_score      0
duration             8
loan_settling        0
dtype: int64

# DATA MODELLING

In [46]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


from sklearn.model_selection import StratifiedShuffleSplit,cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_auc_score

C:\Users\HP ELITE BOOK\Downloads\WinPython-64bit-3.5.4.0Qt5\python-3.5.4.amd64\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [102]:
preds = knn.predict(test_data)
my_submission = pd.DataFrame({'customerid':id,'good_bad_flag':preds.astype(int)})
my_submission.to_csv('sub.csv',index=False)

In [ ]:
preds.astype(int)

In [45]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.boxplot(train_data.age)

In [ ]:
train_data.age[train_data.age >47].value_counts()

In [ ]:
new = []
for val in train_data.age:
    if val>=47:
        new.append(train_data.age.median())
    else:
        new.append(val)

train_data.age = new

In [ ]:
IDcol = train['customerid']

In [60]:
from sklearn.cross_validation import train_test_split
DataX_train , DataX_test, Targety_train, Targety_test = train_test_split(train_data, train_target, random_state = 42, test_size=0.30)

In [61]:
from sklearn import metrics as m
#knn =GradientBoostingClassifier( n_estimators=60,learning_rate=0.1, min_samples_split=500,
     #                                           min_samples_leaf=50,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10,
    #)
knn =GradientBoostingClassifier()
    
knn.fit(DataX_train,Targety_train)
dt2 = knn.predict(DataX_test)
scores=m.accuracy_score(Targety_test,dt2)

In [62]:
scores

0.77650648360030516

In [173]:
print(m.confusion_matrix(Targety_test ,dt2))

[[ 43 249]
 [ 43 976]]


In [63]:
from sklearn import cross_validation,metrics
def modelfit(alg, dtrain, train_target,predictors, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], train_target)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], train_target, cv=cv_folds, scoring='accuracy')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(train_target, dtrain_predictions))
   # print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),
                                                                                  np.max(cv_score)))
                
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [68]:
#Choose all predictors except target & IDcols
predictors = train_data.columns

#gbm0 = GradientBoostingClassifier(n_estimators=50,random_state=10)

modelfit(LogisticRegression(), train_data,train_target,predictors)


Model Report
Accuracy : 0.7816
CV Score : Mean - 0.7820518 | Std - 0.001460278 | Min - 0.7803204 | Max - 0.7846506


AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

In [56]:
from sklearn.grid_search import GridSearchCV
predictors = train_data.columns
est=[i for i in range(20,81,10)]
param_test1 = {'n_estimators':est}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,
                                                               min_samples_leaf=50,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10),
                        param_grid = param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train_target)

C:\Users\HP ELITE BOOK\Downloads\WinPython-64bit-3.5.4.0Qt5\python-3.5.4.amd64\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=500, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [20, 30, 40, 50, 60, 70, 80]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [57]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.78205, std: 0.00039, params: {'n_estimators': 20},
  mean: 0.78182, std: 0.00047, params: {'n_estimators': 30},
  mean: 0.78136, std: 0.00110, params: {'n_estimators': 40},
  mean: 0.78022, std: 0.00293, params: {'n_estimators': 50},
  mean: 0.78045, std: 0.00380, params: {'n_estimators': 60},
  mean: 0.78068, std: 0.00411, params: {'n_estimators': 70},
  mean: 0.77976, std: 0.00453, params: {'n_estimators': 80}],
 {'n_estimators': 20},
 0.7820515708814842)

# MODELLING REVIEW AND CHECK UPS

In [ ]:
train['balance_band'] = pd.cut(train['loanamount'],5)

train[['balance_band', 'good_bad_flag']].groupby(['balance_band'], as_index=False).mean()

In [ ]:
combine = [train_data,test]
for dataset in combine:
    dataset.loc[(dataset['loanamount']<=20000),'loanamount']=1
    dataset.loc[(dataset['loanamount']>20000) & (dataset['loanamount']<=30000),'loanamount']= 2
    dataset.loc[(dataset['loanamount'] >30000) & (dataset['loanamount'] <= 40000),'loanamount'] = 3
    dataset.loc[(dataset['loanamount'] > 40000) & (dataset['loanamount'] <=50000),'loanamount'] = 4
    dataset.loc[dataset['loanamount'] > 50000,'loanamount'] = 5

In [ ]:
train_data.loanamount.value_counts()

In [ ]:
train_data.isnull().sum()

In [ ]:
data_10k =train_data[train_data['loanamount'] == 10000.0]
#data_others = datas[datas['loanamount']!=10000.0]
print (data_10k.shape)
#print (data_others.shape)

In [ ]:
train_targetaa = data_10k.good_bad_flag.values

In [ ]:
data_10k['good_bad_flag'].value_counts()

In [ ]:
data_10k = data_10k.drop(['good_bad_flag'], axis=1)

In [ ]:
data = datas.drop(['good_bad_flag','systemloanid',
                    'referredby','termdays','customerid',], axis=1)

In [ ]:
dataa = pd.get_dummies(data)

In [ ]:
def get_train_test(data):
    
    # Get test set from data
    test_data = data[data['good_bad_flag'].isnull()].reset_index()
    test_data = test_data.drop('index',axis=1)
    
    
    # Get train set from data
    data_train_g = data[data['good_bad_flag'] == 'Good'].reset_index()
    
    # data_10k_train_g = data_10k_train.drop('index',axis=1)

    data_train_b = data[data['good_bad_flag'] == 'Bad'].reset_index()
    # data_10k_train_b = data_10k_train.drop('index',axis=1)

    frames = [data_train_g,data_train_b]
    train_data = pd.concat(frames)

    train_data = train_data.drop('index',axis=1)

In [ ]:
data_10k_train, data_10k_test = get_train_test(data_10k)

In [ ]:
data_others_train, data_others_test = get_train_test(data_others)

In [ ]:
X,Y = get_x_y(data_10k_train)